In [470]:
%run config.py

In [471]:
import random
def rev(qbits,qbit):
	return qbits-qbit-1

def randomgate(qbits):
	op=random.choice(['x','y','z','h','cx','cz','rx','ry','rz'])
	if op=='cx' or op=='cz':
		q1=random.choice(range(qbits))
		q2=random.choice(range(qbits))
		while q1==q2:
			q2=random.choice(range(qbits))
		if op=='cx':
			questOp='controlledNot'
		if op=='cz':
			questOp='controlledPhaseFlip'
		return 'qc.'+op+'('+str(q1)+','+str(q2)+')', op+' q'+str(q1)+',q'+str(q2), questOp+'(qubits, '+str(rev(qbits,q1))+', '+str(rev(qbits,q2))+');'
	elif op=='x' or op=='y' or op=='z' or op=='h':
		q1=random.choice(range(qbits))
		if op=='h':
			questOp='hadamard'
		if op=='x':
			questOp='pauliX'
		if op=='y':
			questOp='pauliY'
		if op=='z':
			questOp='pauliZ'

		return 'qc.'+op+'('+str(q1)+')', op+' q'+str(q1), questOp+'(qubits, '+str(rev(qbits,q1))+');'
	elif op=='rx' or op=='ry' or op=='rz':
		q1=random.choice(range(qbits))
		angle=random.random()*2*3.14159265359
		if op=='rx':
			questOp='rotateX'
		if op=='ry':
			questOp='rotateY'
		if op=='rz':
			questOp='rotateZ'
		return 'qc.'+op+'('+str(angle)+','+str(q1)+')', op+' q'+str(q1)+', '+str(angle), questOp+'(qubits, '+str(rev(qbits,q1))+', '+str(angle)+');'
	 

In [472]:
def randomcircuit(qbits, gates):
	qiscirc=""
	bmcirc=""
	qcirc=""
	for i in range(gates):
		qiscirct, bmcirct, qcirct = randomgate(qbits)
		qiscirc+=qiscirct+'\n'
		bmcirc+='\t'+bmcirct+'\n'
		qcirc+=qcirct+'\n'
	return qiscirc, bmcirc, qcirc

In [473]:
def randomsim(qbits,gates):
	qbitslist=""
	for i in range(qbits):
		if i==0:
			qbitslist+="q"+str(i)
		else:
			qbitslist+=",q"+str(i)
	qiscirc, bmcirc, qcirc = randomcircuit(qbits, gates)
	with open('qiskit/circuit.py.tmpl','r') as f:
		qiskitscript=f.read()

	with open('bmqsim/program.bmq.tmpl','r') as f:
		bmprogram=f.read()

	with open('quest/circuit.c.tmpl','r') as f:
		qprogram=f.read()

	qiskitscript=qiskitscript.replace('%%CIRCUIT%%',qiscirc).replace('%%QBITS%%',str(qbits)).replace('%%QBITSLIST%%',qbitslist)
	bmprogram=bmprogram.replace('%%CIRCUIT%%',bmcirc).replace('%%QBITS%%',str(qbits)).replace('%%QBITSLIST%%',qbitslist)
	qprogram=qprogram.replace('%%CIRCUIT%%',qcirc).replace('%%QBITS%%',str(qbits)).replace('%%QBITSLIST%%',qbitslist)

	with open('qiskit/circuit.py','w') as f:
		f.write(qiskitscript)

	with open('bmqsim/program.bmq','w') as f:
		f.write(bmprogram)

	with open('quest/circuit.c','w') as f:
		f.write(qprogram)

	%run analyze.ipynb

In [474]:
randomsim(qbits,50)
if passed:
	print('Test passed')
else:
	print('Test failed')

     ┌──────────────┐┌───┐   ┌───┐          ┌───┐     ┌────────────┐┌───┐»
q_0: ┤ Ry(0.090333) ├┤ X ├─■─┤ X ├──■───────┤ Z ├─────┤ Rz(4.6623) ├┤ H ├»
     └──────────────┘└─┬─┘ │ ├───┤  │   ┌───┴───┴────┐└───┬───┬────┘├───┤»
q_1: ──────────────────■───■─┤ X ├──┼───┤ Ry(1.7007) ├────┤ X ├─────┤ Y ├»
      ┌────────────┐ ┌───┐   └───┘┌─┴─┐┌┴────────────┤┌───┴───┴────┐├───┤»
q_2: ─┤ Ry(6.0589) ├─┤ Z ├────────┤ X ├┤ Rx(0.56841) ├┤ Rz(3.2426) ├┤ Y ├»
      └────────────┘ └───┘        └───┘└─────────────┘└────────────┘└───┘»
«         ┌───┐                        ┌────────────┐          ┌───┐┌───┐┌───┐»
«q_0: ────┤ H ├─────────────────────■──┤ Rz(1.1452) ├───────■──┤ X ├┤ Z ├┤ Z ├»
«         ├───┤     ┌────────────┐  │  └────────────┘       │  └─┬─┘└───┘└───┘»
«q_1: ────┤ X ├─────┤ Rx(2.2576) ├──┼───────────────────────┼────┼────────────»
«     ┌───┴───┴────┐└────────────┘┌─┴─┐┌────────────┐┌───┐┌─┴─┐  │            »
«q_2: ┤ Rx(3.3066) ├──────────────┤ X ├┤ Rz(3.3107) ├┤ H ├┤ X ├──■─────────